In [ ]:
!pip install qdrant-client
!pip install langchain
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.62.0
    Uninstalling grpcio-1.62.0:
      Successfully uninstalled grpcio-1.62.0
ERROR: pip's dependency resolver does

In [ ]:
# Initialize a retriever using Qdrant and SentenceTransformer embeddings
from langchain.vectorstores import Qdrant
from langchain.retrievers import EnsembleRetriever
from langchain.embeddings import SentenceTransformerEmbeddings
from qdrant_client import QdrantClient
import pandas as pd

embeddings = SentenceTransformerEmbeddings(model_name='sentence-transformers/clip-ViT-B-32')

def get_results(search_results):
    filtered_img_ids = [doc.metadata.get("image_id") for doc in search_results]
    return filtered_img_ids


vector_db_key = 'UOqiBgqhhu8BBWP98mwjGl7h4IhL2vMAqzO4EI9PEB66A50n9GoIiQ'

client = QdrantClient(
    url="https://763bc1da-0673-4535-91ac-b5538ec0287f.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key=vector_db_key,
)

COLLECTION_NAME="semantic_image_search_cleaned"


dense_vector_retriever = Qdrant(client, COLLECTION_NAME, embeddings)
images_data = pd.read_csv("/content/images.csv", on_bad_lines='skip')

def get_link(query):
    Search_Query = query
    neutral_retiever = EnsembleRetriever(retrievers=[dense_vector_retriever.as_retriever()])
    result = neutral_retiever.get_relevant_documents(Search_Query)
    filtered_images = get_results(result)
    filtered_img_ids = [str(doc.metadata.get("image_id")) +'.jpg' for doc in result]

    # links = [images_data.loc[id, 'link'] for id in filtered_img_ids]
    links = [images_data.loc[images_data['filename'] == i, 'link'].values[0] for i in filtered_img_ids]

#     final = '[' + ','.join(links) + ']'
    return  ', '.join(links)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

0_CLIPModel/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

0_CLIPModel/tokenizer_config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

0_CLIPModel/preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

0_CLIPModel/special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
print(get_link("black shirt for wedding").split(","))

['http://assets.myntassets.com/v1/images/style/properties/4817686bd516ebcaef0b9893dfcadb16_images.jpg', ' http://assets.myntassets.com/v1/images/style/properties/6a565cc41bc84507e5d5a3fe07926921_images.jpg', ' http://assets.myntassets.com/v1/images/style/properties/79e693b484848f12265b4dd4a155fae9_images.jpg', ' http://assets.myntassets.com/v1/images/style/properties/912e3ede6d60bfdc43129ed9c5e3da28_images.jpg']


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 11.0 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=ef89ddcaaa69967a184a1f604dd3e123b5bb17c03d9a7937b5679240f34fe51d
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy


In [ ]:
import gradio as gr

def text_to_uppercase(text):
    return get_link(text)

iface = gr.Interface(fn=text_to_uppercase, inputs="text", outputs="text")
iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8c8e6f96c1fe2aefb7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
